In [68]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os

Setting text properties for plots

In [69]:
plt.rcParams.update({'font.size': 26})
plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
plt.close()

Importing dataset

In [70]:
# Specifying simulation directory and the directory to save results in
wdir = str(input('Enter simulation directory path: '))

# Specifying a snapshot for temporal analysis
sdir = wdir + 'temporal_analysis/'

# Create a directory to store correlation plots
if not os.path.exists(sdir + 'correlation_plots/'):
    os.makedirs(sdir + 'correlation_plots/')

Enter simulation directory path: /Users/thepoetoftwilight/Documents/CASSI2020/CASSI2020-Code+Results/Results/m11b_res2100_cr_heating_fix/


In [71]:
# Read in all metals and find store directories to all metals
metal_df = pd.read_csv((sdir + 'metal_list.csv'))
metals = metal_df['metals'].to_list()

spath_metals = {}

for m in metals:
    spath_metals[m] = sdir + m + '/'

In [72]:
# Obtained statistics for the rendered snapshots

rendered_df = pd.read_csv(sdir + 'rendered_snap_stats.csv')

print(rendered_df)

params_df = {}

for m in metals:
    params_df[m] = pd.read_csv(spath_metals[m] + 'data/fit/fit_{}_params.csv'.format(m))

     Unnamed: 0  snap     halo_mass      redshift       time  velocity_mass  \
0             0    80  5.736586e+09  4.338983e+00   1.439067      82.371468   
1             1    85  5.620534e+09  4.121951e+00   1.530909      80.267014   
2             2    90  6.047046e+09  3.923077e+00   1.623934      79.587395   
3             3    95  6.995570e+09  3.740741e+00   1.717761      77.904190   
4             4   100  8.517658e+09  3.571429e+00   1.813212      78.454208   
..          ...   ...           ...           ...        ...            ...   
100         100   580  4.568081e+10  1.782818e-02  13.553508     110.529854   
101         101   585  4.578685e+10  9.646261e-03  13.665295     111.209183   
102         102   590  4.588703e+10  1.594940e-03  13.776556     112.000145   
103         103   595  4.589751e+10  7.968344e-04  13.787654     112.089027   
104         104   600  4.590833e+10  4.440892e-16  13.798747     112.175308   

     velocity_vol  velocity_spread  sound_mass  sou

In [73]:
# Correlating abundance spread of all elements with each other

columns_spread = ['spread_' + m for m in metals]

spread_df = pd.DataFrame(columns = columns_spread)

for m in metals:
    spread_df['spread_' + m] = params_df[m]['std']
    
spread_corr = spread_df.corr()

plt.subplots(figsize = (13, 13))
sns.heatmap(spread_corr, annot = True, square = True)
plt.savefig(sdir + 'correlation_plots/spread_corr.png')
plt.close()

In [74]:
columns_cent = ['central_' + m for m in metals]

cent_df = pd.DataFrame(columns = columns_cent)

for m in metals:
    cent_df['cent_' + m] = params_df[m]['median_mass']
    
cent_corr = cent_df.corr()

plt.subplots(figsize = (13, 13))
sns.heatmap(cent_corr, annot = True, square = True)
plt.savefig(sdir + 'correlation_plots/cent_corr.png')
plt.close()

In [75]:
spread_cent_corr = pd.concat([cent_df, spread_df], axis=1, keys=['cent_df', 'spread_df']).corr().loc['cent_df', 'spread_df']
print(spread_cent_corr)

plt.subplots(figsize = (13, 13))
sns.heatmap(spread_cent_corr, annot = True, square = True)
plt.savefig(sdir + 'correlation_plots/spread_cent_corr.png')
plt.close()

         spread_c  spread_n  spread_o  spread_ne  spread_mg  spread_si  \
cent_c  -0.688352 -0.722107 -0.705659  -0.708627  -0.706182  -0.643572   
cent_n  -0.674698 -0.705816 -0.706360  -0.709543  -0.707105  -0.641784   
cent_o  -0.692363 -0.731397 -0.658790  -0.660771  -0.658319  -0.610502   
cent_ne -0.693524 -0.732276 -0.659660  -0.661596  -0.659200  -0.612189   
cent_mg -0.693187 -0.732088 -0.659983  -0.661961  -0.659527  -0.611814   
cent_si -0.681413 -0.723420 -0.654464  -0.656968  -0.653957  -0.596636   
cent_s  -0.675778 -0.718510 -0.651334  -0.654009  -0.650831  -0.590644   
cent_ca -0.670375 -0.713757 -0.647738  -0.650532  -0.647233  -0.584934   
cent_fe -0.629770 -0.674621 -0.622566  -0.626151  -0.622155  -0.547417   

         spread_s  spread_ca  spread_fe  
cent_c  -0.623057  -0.602343  -0.509085  
cent_n  -0.620431  -0.598979  -0.498348  
cent_o  -0.594488  -0.577832  -0.518983  
cent_ne -0.596450  -0.580062  -0.522151  
cent_mg -0.595845  -0.579238  -0.520362  
cent_si

In [76]:
fe_columns = ['spread_fe', 'cent_fe']
fe_df = pd.DataFrame({'spread_fe': params_df['fe']['std'], 'cent_fe': params_df['fe']['median_mass']})

In [77]:
SFR_df = rendered_df[['SFR@10Myr', 'SFR@100Myr', 'SFR@1000Myr']].copy()

SFR_chem_corr = pd.concat([fe_df, SFR_df], axis=1, keys=['fe_df', 'SFR_df']).corr().loc['fe_df', 'SFR_df']
print(SFR_chem_corr)

plt.subplots(figsize = (13, 13))
sns.heatmap(SFR_chem_corr, annot = True, square = True)
plt.savefig(sdir + 'correlation_plots/SFR_fe_corr.png')
plt.close()

           SFR@10Myr  SFR@100Myr  SFR@1000Myr
spread_fe   0.274170    0.140103     0.342408
cent_fe    -0.414053   -0.511329    -0.872767


In [78]:
halo_df = rendered_df[['halo_mass']].copy()

halo_chem_corr = pd.concat([fe_df, halo_df], axis=1, keys=['fe_df', 'halo_df']).corr().loc['fe_df', 'halo_df']
print(halo_chem_corr)

plt.subplots(figsize = (13, 13))
sns.heatmap(halo_chem_corr, annot = True, square = True)
plt.savefig(sdir + 'correlation_plots/halo_fe_corr.png')
plt.close()

           halo_mass
spread_fe  -0.197443
cent_fe     0.886008


In [79]:
velo_df = rendered_df[['velocity_mass', 'sound_mass', 'thermal_mass']].copy()

velo_chem_corr = pd.concat([fe_df, velo_df], axis=1, keys=['fe_df', 'velo_df']).corr().loc['fe_df', 'velo_df']
print(velo_chem_corr)

plt.subplots(figsize = (13, 13))
sns.heatmap(velo_chem_corr, annot = True, square = True)
plt.savefig(sdir + 'correlation_plots/velo_fe_corr.png')
plt.close()

           velocity_mass  sound_mass  thermal_mass
spread_fe       0.351429   -0.042216     -0.042215
cent_fe         0.523551   -0.763940     -0.763939


In [80]:
mach_df = rendered_df[['mach_number_mass']].copy()

mach_chem_corr = pd.concat([fe_df, mach_df], axis=1, keys=['fe_df', 'mach_df']).corr().loc['fe_df', 'mach_df']
print(mach_chem_corr)

plt.subplots(figsize = (13, 13))
sns.heatmap(mach_chem_corr, annot = True, square = True)
plt.savefig(sdir + 'correlation_plots/mach_fe_corr.png')
plt.close()

           mach_number_mass
spread_fe          0.208558
cent_fe            0.668811
